In [2]:
import os
import csv
import pandas as pd
import numpy as np
import librosa
import plotly.express as px
from datetime import datetime
from maad import sound, features, util
from pprint import pprint
from pathlib import Path

from birdnetlib import Recording
from birdnetlib.analyzer import Analyzer
from birdnetlib.batch import DirectoryMultiProcessingAnalyzer

## Renaming the files so they can be read by birdnet
Birdnet only accepts ".wav" files, the ones provided by the Audiomoths are ".WAV" files. We have to rename the files so birdnet can use them 

In [ ]:

# === Paramètre ===
DATA_PATH = "./Ecoacoustics_Longitudinal_Altitude_Project/"  # dossier racine à traiter

# Parcours récursif de tous les sous-dossiers
for root, dirs, files in os.walk(DATA_PATH):
    for file_name in files:
        # Vérifie si le fichier a une extension WAV (majuscules ou mixtes)
        if file_name.lower().endswith(".wav"):
            old_path = os.path.join(root, file_name)
            # Crée le nouveau nom avec extension en minuscules
            base_name = os.path.splitext(file_name)[0]
            new_path = os.path.join(root, base_name + ".wav")
            # Renomme si le nom est différent
            if old_path != new_path:
                os.rename(old_path, new_path)
                print(f"Renommé : {old_path} -> {new_path}")


## Applying birdnet algorithm to ONE folder

In [49]:
# === Paramètres ===
DATA_PATH = "./Ecoacoustics_Longitudinal_Altitude_Project/Point_1"  # dossier contenant tous les fichiers .wav
OUTPUT_CSV = "birdnet_detections.csv"    # fichier CSV final
EXPORT_DIR = "extractions"               # dossier pour audio/spectrogrammes
LON = 6.07342607201401
LAT = 46.5104596567472
MIN_CONF = 0.2
ANALYSIS_DATE = datetime(year=2025, month=10, day=1)

# Crée le dossier d'export si nécessaire
os.makedirs(EXPORT_DIR, exist_ok=True)

# Initialise l'analyseur BirdNET
analyzer = Analyzer()

# Prépare le CSV
csv_file = open(OUTPUT_CSV, mode="w", newline="")
fieldnames = [
"directory", "file_name", "species", "confidence",
    "start_time", "end_time", "lat", "lon", "date"
]
writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
writer.writeheader()

# Fonction appelée à la fin du traitement du dossier
def on_analyze_directory_complete(recordings):
    print("-" * 80)
    print("directory_completed: recordings processed ", len(recordings))
    print("-" * 80)

    for recording in recordings:
        print(recording.path)
        if recording.error:
            print("Error: ", recording.error_message)
        else:
            # Export audio et spectrogrammes
            recording.extract_detections_as_audio(directory=EXPORT_DIR)
            recording.extract_detections_as_spectrogram(directory=EXPORT_DIR)

            # Écrit les détections dans le CSV
            for det in recording.detections:
                writer.writerow({
                    "directory": os.path.basename(os.path.dirname(recording.path)),
                    "file_name": os.path.basename(recording.path),
                    "species": det["common_name"],
                    "confidence": det["confidence"],
                    "start_time": det["start_time"],
                    "end_time": det["end_time"],
                    "lat": LAT,
                    "lon": LON,
                    "date": recording.date.strftime("%Y-%m-%d"),
                })
            pprint(recording.detections)
        print("-" * 80)

# Crée le batch multi-processus
batch = DirectoryMultiProcessingAnalyzer(
    DATA_PATH,
    analyzers=[analyzer],
    lon=LON,
    lat=LAT,
    date=ANALYSIS_DATE,
    min_conf=MIN_CONF,
)

# Remplace la fonction callback pour gérer le CSV
batch.on_analyze_directory_complete = on_analyze_directory_complete

# Lance l'analyse
batch.process()

# Ferme le CSV
csv_file.close()

print(f"✅ Analyse terminée. CSV généré : {OUTPUT_CSV}")


Labels loaded.
load model True
Model loaded.
Labels loaded.
load_species_list_model
Meta model loaded.


/home/boulbi/Bureau/COURS/MoBi/ecoacoustics/.venv/lib/python3.13/site-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


process_from_queueprocess_from_queue
process_from_queue
Initializing analyzer(s)
Initializing analyzer(s)process_from_queueInitializing analyzer(s)



Initializing analyzer(s)
process_from_queueLabels loaded.Labels loaded.Labels loaded.
load modelprocess_from_queue
process_from_queue
Labels loaded.Initializing analyzer(s)

 
load model
load modelTrue load model
Initializing analyzer(s)Initializing analyzer(s) 
 
True
True
True
Labels loaded.

load modelLabels loaded.Labels loaded.
 Trueload model

load model  TrueTrue

Model loaded.Model loaded.
Model loaded.
Model loaded.

Labels loaded.Labels loaded.Labels loaded.


Labels loaded.Model loaded.load_species_list_modelload_species_list_model

load_species_list_model
Model loaded.load_species_list_model



Labels loaded.
Model loaded.Labels loaded.load_species_list_model

Labels loaded.

load_species_list_modelload_species_list_model

Meta model loaded.Meta model loaded.

read_audio_dataread_audio_data

Meta model loaded.Meta model loade

## Applying Birdnet to one folder and the sub-folders

In [1]:
# === Paramètres ===
DATA_PATH = "./Ecoacoustics_Longitudinal_Altitude_Project"  # dossier racine
OUTPUT_CSV = "birdnet_detections_2.csv"    # fichier CSV final
EXPORT_DIR = "extractions"               # dossier pour audio/spectrogrammes
LON = 6.07342607201401
LAT = 46.5104596567472
MIN_CONF = 0.4
ANALYSIS_DATE = datetime(year=2025, month=10, day=1)
# GAIN_DB = 6  # gain à appliquer en dB (mettre 0 pour pas toucher)

# Crée le dossier d'export si nécessaire
os.makedirs(EXPORT_DIR, exist_ok=True)

# Initialise l'analyseur BirdNET
analyzer = Analyzer()

# Prépare le CSV
csv_file = open(OUTPUT_CSV, mode="w", newline="")
fieldnames = [
    "directory", "file_name", "species", "confidence",
    "start_time", "end_time", "lat", "lon", "date"
]
writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
writer.writeheader()

# Fonction appelée à la fin du traitement du dossier
def on_analyze_directory_complete(recordings):
    print("-" * 80)
    print("directory_completed: recordings processed ", len(recordings))
    print("-" * 80)

    for recording in recordings:
        print(recording.path)
        if recording.error:
            print("Error: ", recording.error_message)
        else:
            # Export audio et spectrogrammes
            recording.extract_detections_as_audio(directory=EXPORT_DIR)
            recording.extract_detections_as_spectrogram(directory=EXPORT_DIR)

            # Écrit les détections dans le CSV
            for det in recording.detections:
                writer.writerow({
                    "directory": os.path.basename(os.path.dirname(recording.path)),
                    "file_name": os.path.basename(recording.path),
                    "species": det["common_name"],
                    "confidence": det["confidence"],
                    "start_time": det["start_time"],
                    "end_time": det["end_time"],
                    "lat": LAT,
                    "lon": LON,
                    "date": recording.date.strftime("%Y-%m-%d"),
                })
            pprint(recording.detections)
        print("-" * 80)

# Crée le batch multi-processus sur la liste de fichiers
for folder in os.listdir(DATA_PATH):
    full_path = os.path.join(DATA_PATH, folder)
    print(f"folder : {folder}")
    if os.path.isdir(full_path):
        print(f"Analyse de {folder} ...")
        batch = DirectoryMultiProcessingAnalyzer(
            full_path,          # liste de fichiers
            analyzers=[analyzer],
            lon=LON,
            lat=LAT,
            date=ANALYSIS_DATE,
            min_conf=MIN_CONF,
        )
        # Lance l'analyse
        batch.on_analyze_directory_complete = on_analyze_directory_complete
        batch.process()

# Ferme le CSV
csv_file.close()

print(f"✅ Analyse terminée. CSV généré : {OUTPUT_CSV}")


Labels loaded.
load model True
Model loaded.
Labels loaded.
load_species_list_model
Meta model loaded.
folder : readMe.txt
folder : Point_7
Analyse de Point_7 ...


/home/boulbi/Bureau/COURS/MoBi/ecoacoustics/.venv/lib/python3.13/site-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


process_from_queueprocess_from_queue

Initializing analyzer(s)process_from_queueprocess_from_queue

Initializing analyzer(s)Initializing analyzer(s)

Initializing analyzer(s)process_from_queue
Labels loaded.
Labels loaded.


load modelprocess_from_queueLabels loaded.Labels loaded.load modelInitializing analyzer(s) 

 
Initializing analyzer(s)Trueload model
process_from_queue load modelTrue
TrueLabels loaded.


 
Labels loaded.

Initializing analyzer(s)load model
load modelTrue True Labels loaded.


Trueload model True

Model loaded.
Model loaded.Model loaded.

Labels loaded.
load_species_list_modelLabels loaded.Model loaded.Labels loaded.


Model loaded.
load_species_list_modelload_species_list_model
Model loaded.

Labels loaded.Labels loaded.


Model loaded.
load_species_list_modelload_species_list_modelMeta model loaded.Labels loaded.



Labels loaded.Meta model loaded.read_audio_data

load_species_list_model
read_audio_data
load_species_list_model

Meta model loaded.
read_audio_data

/home/boulbi/Bureau/COURS/MoBi/ecoacoustics/.venv/lib/python3.13/site-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


Labels loaded.
load model True


...

/home/boulbi/Bureau/COURS/MoBi/ecoacoustics/.venv/lib/python3.13/site-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


 True

/home/boulbi/Bureau/COURS/MoBi/ecoacoustics/.venv/lib/python3.13/site-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


/home/boulbi/Bureau/COURS/MoBi/ecoacoustics/.venv/lib/python3.13/site-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


Model loaded.
Model loaded.
Model loaded.Labels loaded.
Labels loaded.

load_species_list_modelLabels loaded.Model loaded.Model loaded.Model loaded.


load_species_list_model

load_species_list_model
Labels loaded.Labels loaded.
Model loaded.

load_species_list_modelload_species_list_model
Labels loaded.


load_species_list_model
Labels loaded.
load_species_list_model
Meta model loaded.
read_audio_data
Meta model loaded.
Meta model loaded.read_audio_data

Meta model loaded.read_audio_dataMeta model loaded.

Meta model loaded.

read_audio_dataread_audio_dataread_audio_data


Meta model loaded.
read_audio_dataread_audio_data: complete, read 
 20read_audio_data: complete, read read_audio_data: complete, read   20  20chunks.read_audio_data: complete, read  chunks. read_audio_data: complete, read 
read_audio_data: complete, read 20chunks.
  analyze_recording
analyze_recording  2020 chunks.20251001_042000.wav analyze_recording 20251001_122000.wav
 
chunks.chunks.
20251001_142000.wavrecording

## Data analysis

### Load Data from csv if not already available

In [21]:
df_detection = pd.read_csv('birdnet_detections_with_gps.csv')

### Utils for data shaping

In [22]:

def extract_datetime_from_filename(filename):
    try:
        # On isole le nom sans extension
        base = Path(filename).stem  # ex: "20231027_142533_recording"
        # On prend les deux premières parties séparées par "_"
        parts = base.split('_')
        if len(parts) < 2:
            return None  # format inattendu
        
        date_str, heure_str = parts[0], parts[1]
        # Conversion en datetime
        return datetime.strptime(date_str + heure_str, "%Y%m%d%H%M%S")
    except Exception as e:
        print(f"⚠️ Erreur pour {filename}: {e}")
        return None

# Application au DataFrame

def set_id_from_filename(df):
    df["id"] = df["directory"] + "_" + df["file_name"].str.split('.').str[0]
    return df

def set_gps_from_directory(df, gps_coord):

    df["lat"] = df["directory"].str.split('_').str[1].astype(int).map(gps_coord.set_index('dossier')['latitude'])
    df["lon"] = df["directory"].str.split('_').str[1].astype(int).map(gps_coord.set_index('dossier')['longitude'])
    df["alt"] = df["directory"].str.split('_').str[1].astype(int).map(gps_coord.set_index('dossier')['altitude'])

    return df

### DataFrame shaping (not necessarily needed if dataframe already correct)

In [24]:
dossier_racine = r"./Ecoacoustics_Longitudinal_Altitude_Project"

df_detection["datetime"] = df_detection["file_name"].apply(extract_datetime_from_filename)
df_detection = set_id_from_filename(df_detection)

# gps_coord = pd.read_csv(dossier_racine+'/gps_coord.csv')  # Charger les coordonnées GPS si nécessaire

# df_detection = set_gps_from_directory(df_detection, gps_coord)

### Data exploration

#### A few useful lines if you want to check your dataframe

In [15]:
df_detection.columns # will show all column names

df_detection.head() # will show the first 5 rows of the dataframe

df_detection["species"].unique() # will show unique species names
df_detection["species"].nunique() # will show the number of unique species

df_detection["confidence"].describe() # will show statistics of confidence scores

df_detection.shape # will show the shape of the dataframe (rows, columns)

df_detection.size # will show the total number of elements in the dataframe

df_detection.groupby(['directory']).size()  # will show the number of detections per directory

df_detection.groupby(['species']).size().sort_values(ascending=False)  # will show the number of detections per species, sorted descending



,directory,file_name,species,confidence,start_time,end_time,lat,lon,date,altitude,id
0,Point_1,20250930_171000.wav,Eurasian Bullfinch,0.579799,33.0,36.0,"6,07342607201401","46,5104596567472",2025-09-30 17:10:00,1250,Point_1_20250930_171000
1,Point_1,20250930_173000.wav,Eurasian Bullfinch,0.766247,42.0,45.0,"6,07342607201401","46,5104596567472",2025-09-30 17:30:00,1250,Point_1_20250930_173000
2,Point_1,20250930_175000.wav,Common Chaffinch,0.453650,0.0,3.0,"6,07342607201401","46,5104596567472",2025-09-30 17:50:00,1250,Point_1_20250930_175000
3,Point_1,20250930_175000.wav,Eurasian Bullfinch,0.470017,15.0,18.0,"6,07342607201401","46,5104596567472",2025-09-30 17:50:00,1250,Point_1_20250930_175000
4,Point_1,20250930_175000.wav,Eurasian Bullfinch,0.463013,24.0,27.0,"6,07342607201401","46,5104596567472",2025-09-30 17:50:00,1250,Point_1_20250930_175000


#### A few useful lines to shape your dataframe

In [16]:
df_detection.sort_values(by=['altitude', 'datetime'], inplace=True) # Sort by altitude and datetime

directory
Point_1      89
Point_10    201
Point_11    115
Point_12     52
Point_3      35
Point_4     170
Point_5     152
Point_6      61
Point_7     148
Point_8     160
Point_9     251
dtype: int64

### Basic plot

In [30]:
fig = px.histogram(
    df_detection,
    x='datetime',
    color='alt',
    # nbins=50,
    title='Histogramme des détections BirdNet au fil du temps',
    barmode='group', 
    histnorm=''
)

fig.show()

### Compute the number of microphones per altitude (in order to be able to compare the results)

In [27]:
df_detec_norm = (
    df_detection
    .groupby(["alt", "datetime"])
    .size()
    .rename("counts")
    .reset_index()
)

# Ajout du nombre de micros par altitude
mic_counts = df_detection.groupby("alt")["directory"].nunique().rename("mic_count")
df_detec_norm = df_detec_norm.merge(mic_counts, on="alt", how="left")

# Normalisation par micro
df_detec_norm["norm_counts"] = df_detec_norm["counts"] / df_detec_norm["mic_count"]

### Figure

In [29]:
fig = px.histogram(
    df_detec_norm,
    x="datetime",
    y="norm_counts",
    color="alt",
    title="Détections BirdNET normalisées par le nombre de micros",
    labels={"norm_counts": "Détections / micro"},
    nbins=25,
    barmode='group',
)
fig.show()

### A few lines to analyze our birbs

In [ ]:
df_detection.groupby(['alt']).size() # will show the number of detections per altitude


species_df = df_detection.groupby(['species'], as_index=False)['file_name'].count() # creates a DataFrame with species and their detection counts
species_df.rename(columns={'file_name': 'detection_count'}, inplace=True)  # rename column for clarity
species_df["mean_confidence"] = df_detection.groupby('species')['confidence'].mean().values # calculate mean confidence per species
species_df["mean_confidence"] = df_detection.groupby('species')['confidence'].median().values # calculate median confidence per species
species_df.sort_values(by='detection_count', ascending=False, inplace=True) # sort by detection count descending

sorted_species_df = species_df[species_df["mean_confidence"] >0.5] # filter species with mean confidence > 0.5


In [40]:
specific_richness_df = df_detection.groupby(['alt', 'datetime'])['species'].nunique().reset_index() # count unique species per altitude and datetime

In [44]:
fig = px.histogram(
    specific_richness_df,
    x="datetime",
    y="species",
    color="alt",
    title="Specific richness per altitude per datetime",
    # labels={"norm_counts": "Détections / micro"},
    nbins=25,
    barmode='group',
)
fig.show()